In [1]:
#%% Imports
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
import pandas as pd
from datetime import datetime
import time

In [2]:
# Another way of authenticating using the Spotipy Library
def authenticate_spotify():
    """
    Uses environment variables to gain access to Spotify API
    
    Environment variables:
    SPOTIPY_CLIENT_ID
    SPOTIPY_CLIENT_SECRET
    SPOTIPY_REDIRECT_URI

    All found on the Spotify Developers Dashboard:
    https://developer.spotify.com/dashboard/20f128d8178c4ba3b89f4adf4e495b44/settings
    
    Returns:
        spotipy.Spotify instance that interacts with Spotify API
    """
    scope = ['playlist-read-private', 'user-read-recently-played', 'user-top-read', 'user-library-read']
    spotify = spotipy.Spotify(auth_manager=SpotifyOAuth(
        client_id=os.getenv('SPOTIPY_CLIENT_ID'),
        client_secret=os.getenv('SPOTIPY_CLIENT_SECRET'),
        redirect_uri=os.getenv('SPOTIPY_REDIRECT_URI'),
        scope=scope))
    
    return spotify

spotify = authenticate_spotify()
spotify

In [7]:
def get_users_top_tracks(spotify, time_range, offset=0):
    tracks = []
    top_tracks = spotify.current_user_top_tracks(limit=50, offset=offset, time_range=time_range)
    print('check')
    
    for item in top_tracks['items']:
        song_name = item['name']
        print(song_name)
        album_name = item['album']['name']
        release_date = item['album']['release_date']
        first_artist_id = item['artists'][0]['id']
        first_artist = spotify.artist(first_artist_id)
        first_artist_name = first_artist['name']
        artists = []
        for artist in item['artists']:
            artists.append(artist['name'])
        genres = first_artist['genres']
        length = item['duration_ms']
        popularity = item['popularity']
        explicit = item['explicit']
        
        

        tracks.append([song_name, artists, album_name, genres, length, popularity, release_date, explicit])
    return tracks


In [9]:
top_tracks = get_users_top_tracks(spotify, time_range='long_term')
top_tracks_df = pd.DataFrame(top_tracks, columns=['Title', 'Artists', 'Album', 'Genres', 'Length (ms)', 'Popularity', 'Release Date', 'Explicit'])
top_tracks_df.head()

check
I'm In Here
Einaudi: Experience
Zimmer
Taranta
Ferma Zitella
WECK MICH AUF
enough for you
Survive
Tränen aus Kajal
26
Kugeln im Lauf
good 4 u
Youth
Another Love
Where's My Love
Levitating
Veuve
All For You
Crime of Passion (Piano Solo Version) [From "Preston Tylk"]
Stoned at the Nail Salon
brutal
Aufstehn! (feat. CeeLo Green)
It's Ok
jealousy, jealousy
Una Mattina
Ich glaub ich will heut nicht mehr gehen
ONLY YOU (feat. Dhurata Dora)
Choros
Stamp On The Ground
Flora
Sleep When We're Dead - Video Edit
Saved My Life
Take Me To Church
Half A Man
I Can't Go on Without You
Taranta
Caribbean Queen (No More Love On the Run)
Seven Days Walking / Day 1: The Path Of The Fossils
Little Lighter
Vergesse nie die Street
Wonderful Life
17 für immer
Nie verliebt
Cello Suite No. 1 in G Major, BWV 1007: I. Prélude
You Keep Me Hangin On
Seven Days Walking / Day 1: The Path Of The Fossils
Introductio Ad Regnum Tarantulae
schwarze SUVs
WECK MICH AUF - HELLMIX
Cry (feat. Grimes)


,Title,Artists,Album,Genres,Length (ms),Popularity,Release Date,Explicit
0,I'm In Here,[Sia],We Are Born,"[australian dance, australian pop, pop]",221320,50,2010-06-18,False
1,Einaudi: Experience,"[Ludovico Einaudi, Daniel Hope, I Virtuosi Ita...",In A Time Lapse (Deluxe Edition),"[compositional ambient, neo-classical]",315426,55,2013-01-01,False
2,Zimmer,[Provinz],Zu spät um umzudrehen - EP,"[german indie folk, german pop]",177818,57,2021-06-11,False
3,Taranta,"[Canzoniere Grecanico Salentino, Ludovico Eina...",Quaranta,"[italian folk, musica etnica, tarantella]",187146,29,2015-04-14,False
4,Ferma Zitella,"[Traditional, Ludovico Einaudi]",Taranta Project,"[gregorian chant, hengelliset laulut, puirt-a-...",245987,29,2015-04-21,False


In [21]:
top_tracks_df.to_csv('top_tracks_long_term.csv', encoding='utf-8-sig')

In [13]:
def convert_datetime_string_to_milliseconds(datetime_str):
    """
    Converts a datetime string in the format '%Y-%m-%dT%H:%M:%S.%fZ' to milliseconds.

    Args:
        datetime_str (str): The datetime string to convert.

    Returns:
        int: The datetime converted to milliseconds.
    """
    datetime_obj = datetime.strptime(datetime_str, '%Y-%m-%dT%H:%M:%S.%fZ')
    milliseconds = int(time.mktime(datetime_obj.timetuple()) * 1000)
    return milliseconds

def fetch_last_function_call():
    with open('recently_played_songs.csv', 'r', errors='ignore') as csvfile:
        last_row = csvfile.readlines()[-1]
        
    date_time_string = last_row.split(',')[-1].replace('\n', '')
    last_function_call_ms = convert_datetime_string_to_milliseconds(date_time_string)
    return last_function_call_ms, date_time_string

In [64]:
with open('recently_played_songs.csv', 'r', errors='ignore') as csvfile:
        last_row = csvfile.readlines()[-1]
print(last_row)
date_time_string = last_row.split(',')[-1].replace('\n', '')
date_time_string

Mann vom Fach,Levin Liam,Mann vom Fach,[],160813,64,2023-10-06,playlist,False,2023-12-13T11:21:25.234Z



'2023-12-13T11:21:25.234Z'

In [53]:
after, date_time_string = fetch_last_function_call()
date_time_string

'2023-12-13T11:13:03.990Z'

In [14]:
def get_recently_played_songs(spotify):
    tracks = []
    after, date_time_string = fetch_last_function_call()
    recently_played_songs = spotify.current_user_recently_played(limit=50, after=after)
    # print('check')
        
    for item in recently_played_songs['items']:
        song_name = item['track']['name']
        print(song_name)
        played_at = item['played_at'] 
        if played_at == date_time_string:
            print('    Reached last song of previous call. Stopping here...')
            break
        album_name = item['track']['album']['name']
        release_date = item['track']['album']['release_date']
        first_artist_id = item['track']['artists'][0]['id']
        first_artist = spotify.artist(first_artist_id)
        first_artist_name = first_artist['name']
        artists = []
        for artist in item['track']['artists']:
            artists.append(artist['name'])
        genres = first_artist['genres']
        length = item['track']['duration_ms']
        popularity = item['track']['popularity']
        try:
            context = item['context']['type']
        except TypeError:
            context = 'saved'
        explicit = item['track']['explicit']

        tracks.append([song_name, artists, album_name, genres, length, popularity, release_date, context, explicit, played_at])

    return tracks

In [15]:
recently_played_songs = get_recently_played_songs(spotify)

Death by a Thousand Cuts
Death by a Thousand Cuts
Empty
schwarze SUVs
Schmetterling
Hör mal mein Herz <3…
Du hast mich nie geliebt </3…
Fake As Hell (with Avril Lavigne)
Fake As Hell (with Avril Lavigne)
deep fake
Psycho
Psycho
DArkSide
Heartbreak Feels So Good
War
Lost Without You
Venom
Fantasy
Reckless & Relentless - Document One Remix
Fuck (feat. Josh Franceschi)
Bite My Tongue
Chicken Bone
Another You
Vice Grip
High For This - Original
Ich hass dich
DER LETZTE SONG (ALLES WIRD GUT) (feat. Nina Chuba)
Pray For Me
Insomnia (feat. Giant Rooks)
Playing God
Stature
Stature
Nie verliebt
Singing Low
XO
Muévelo
    Reached last song of previous call. Stopping here...


In [16]:
recently_played_songs_df = pd.DataFrame(recently_played_songs, columns=['Title', 'Artists', 'Album', 'Genre', 'Length (ms)', 'Popularity', 'Release Date', 'Context', 'Explicit', 'Played at'])
recently_played_songs_df.sort_values('Played at', axis=0, ascending=True, inplace=True)
recently_played_songs_df.head()

,Title,Artists,Album,Genre,Length (ms),Popularity,Release Date,Context,Explicit,Played at
34,XO,[EDEN],i think you think too much of me,[indie poptimism],159512,66,2016-08-19,saved,False,2023-12-13T17:55:12.377Z
33,Singing Low,[The Fray],Through the Years: The Best of The Fray,"[neo mellow, piano rock, pop, pop rock]",263724,32,2016-11-04,saved,False,2023-12-13T17:59:23.234Z
32,Nie verliebt,[Paula Hartmann],Nie verliebt,[],212661,59,2022-08-26,saved,False,2023-12-13T23:48:20.463Z
31,Stature,"[Drama B, Besomorph]",Stature,"[deep underground hip hop, traprun]",182697,34,2018-11-23,saved,False,2023-12-13T23:48:30.336Z
30,Stature,"[Drama B, Besomorph]",Stature,"[deep underground hip hop, traprun]",182697,34,2018-11-23,saved,False,2023-12-14T11:46:35.379Z


In [20]:
recently_played_songs_df.to_csv('recently_played_songs.csv', mode='a', encoding='utf-8-sig', header=False, index=False)

In [22]:
def get_users_saved_tracks(spotify, offset):
    tracks = []
    saved_songs = spotify.current_user_saved_tracks(limit=50, offset=offset)
    print('check! Offset =', offset)
        
    for item in saved_songs['items']:
        song_name = item['track']['name']
        print(song_name)
        album_name = item['track']['album']['name']
        first_artist_id = item['track']['artists'][0]['id']
        first_artist = spotify.artist(first_artist_id)
        first_artist_name = first_artist['name']
        artists = []
        for artist in item['track']['artists']:
            artists.append(artist['name'])
        genres = first_artist['genres']
        length = item['track']['duration_ms']
        popularity = item['track']['popularity']
        added_at = item['added_at']
        release_date = item['track']['album']['release_date']
        explicit = item['track']['explicit']
        

        tracks.append([song_name, artists, album_name, genres, length, popularity, release_date, added_at, explicit])

            
    return tracks

In [23]:
max_number_of_tracks = 3000
all_saved_tracks = []
offset = 0
while offset < max_number_of_tracks:
    saved_tracks = get_users_saved_tracks(spotify, offset=offset)
    all_saved_tracks += saved_tracks
    offset += 50


check! Offset = 0
Empty
Fake As Hell (with Avril Lavigne)
love is embarrassing
L’enfer
Santé
Formidable
Tous les mêmes
Give It Up to Me
G.O.A.T.
Playing God
Seeing Red
Gekreuzte Finger
Cry (feat. Grimes)
Miss Nectarine
Warum lügst du?
Forever & Ever More
Save Yourself
Break My Fall
Happier Than Ever
Honey
To Me It Was
Is There Something in the Movies?
Big Wheel
Death by a Thousand Cuts
Lost
Mirrors - Hypertechno Remix
Babyblau - Remix
Typa Girl
Herzcrash
Schmetterling
Hör mal mein Herz <3…
Ohne Benzin
KETA UND KRAWALL
Maldosa
MALDITA PERIGOSA
Maldita
MANO DO AK
Let's Go!
Devils Price
Halloweenie V: The Moss King
R U Mine?
Paint
Energies
Rehab
Machines
DArkSide
PARANOID
Vois sur ton chemin - Techno Mix
Dead Inside
Nie verliebt
check! Offset = 50
Beautiful Escape
Superstar (feat. Remington Leith)
Kugeln im Lauf
Veuve
schwarze SUVs
Danke Nein
Wenn Du weinst
De Selby (Part 2)
Kein Happy End
Eat Your Young
Put It on Me
Tauchen
I Can't Go on Without You
To Be Alone
Gramm für Gramm
Levitating

In [24]:
len(all_saved_tracks)

2076

In [25]:
all_saved_tracks_df = pd.DataFrame(all_saved_tracks, columns=['Title', 'Artists', 'Album', 'Genre', 'Length (ms)', 'Popularity', 'Release Date', 'Added at', 'Explicit'])
all_saved_tracks_df.head()

,Title,Artists,Album,Genre,Length (ms),Popularity,Release Date,Added at,Explicit
0,Empty,[Letdown.],Crying In The Shower,[],213268,59,2023-02-24,2023-12-15T09:17:18Z,False
1,Fake As Hell (with Avril Lavigne),"[All Time Low, Avril Lavigne]",Fake As Hell (with Avril Lavigne),"[modern rock, neon pop punk, pop emo, pop punk]",179166,68,2023-09-15,2023-12-14T22:37:06Z,True
2,love is embarrassing,[Olivia Rodrigo],GUTS,[pop],154516,82,2023-09-08,2023-12-11T22:39:25Z,True
3,L’enfer,[Stromae],Multitude,"[belgian pop, g-house]",189720,54,2022-07-27,2023-12-10T12:46:33Z,False
4,Santé,[Stromae],Multitude,"[belgian pop, g-house]",191000,55,2022-07-27,2023-12-10T12:46:28Z,False


In [26]:
all_saved_tracks_df.shape

(2076, 9)

In [27]:
all_saved_tracks_df.to_csv('saved_tracks_15_12_2023.csv', encoding='utf-8-sig')